In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


In [2]:

# Load and preprocess the data
data = []
labels = []
folder_path = 'leaf'

for species_folder in os.listdir(folder_path):
    species_path = os.path.join(folder_path, species_folder)
    for img_file in os.listdir(species_path):
        img_path = os.path.join(species_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (100, 100))  # Resize the image to a consistent size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        img = img/255.0  # Normalize
        data.append(img.flatten())  # Flatten the image into a 1D array
        labels.append(species_folder)

data = np.array(data)
labels = np.array(labels)


In [3]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
# Define the SVM Model
svm_model = SVC(probability=True)


In [5]:
# Parameter Tuning using GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf','poly']}
grid_search = GridSearchCV(svm_model, param_grid, cv=5, verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

print(best_params)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   9.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   8.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   8.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   8.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   8.4s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   2.9s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   3.0s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   3.0s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   2.9s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   2.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   8.4s
[CV] END .......................C=0.1, gamma=0.

In [6]:
# Train model using best params
svm_model = SVC(**best_params)
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

                      precision    recall  f1-score   support

                Acer       0.89      0.94      0.91        17
        Alnus incana       0.82      1.00      0.90        18
    Betula pubescens       0.81      0.76      0.79        17
     Fagus silvatica       0.92      0.92      0.92        13
             Populus       0.89      0.89      0.89         9
     Populus tremula       1.00      0.88      0.94        17
             Quercus       1.00      0.87      0.93        15
Salix alba 'Sericea'       1.00      0.94      0.97        16
        Salix aurita       0.62      0.62      0.62        13
       Salix sinerea       0.74      0.89      0.81        19
    Sorbus aucuparia       1.00      1.00      1.00        14
   Sorbus intermedia       0.94      0.94      0.94        17
               Tilia       1.00      0.92      0.96        13
  Ulmus carpinifolia       0.72      0.72      0.72        18
        Ulmus glabra       0.71      0.56      0.62         9

      

In [7]:
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

The model is 86.66666666666667% accurate
